CONTAGION 
NODE BLOCKING
ERDOS-RENYI GRAPH


In [1]:
import matplotlib.pyplot as plt
from networkx import nx
import numpy as np
import algorithmx
import random
import pandas as pd
import os


list_n=[100,500]
list_m=[400,2000]
# percentage listler büyükten küçüğe olmak zorunda!!!!
infected_percentage_list = [0.15,0.1,0.05]
risky_percentage_list = [0.6,0.5,0.4,0.3]
name = "utku"


### node features klasörü açmayı unutma
### node features klasörü açmayı unutma
for elem in range(len(list_n)):
    random.seed(elem)
    n=list_n[elem]
    m=list_m[elem]
    G = nx.gnm_random_graph(n, m)
    colnames=range(1,(n+1))
    adjacency_df = pd.DataFrame(np.zeros((n,n)),columns=colnames,index=colnames)

    for (u, v) in G.edges():
        G.edges[u,v]['weight'] = np.random.randint(0,1000)/1000
        adjacency_df.iat[u,v]=G.edges[u,v]['weight']
        adjacency_df.iat[v,u]=G.edges[u,v]['weight']
    adjacency_df['Threshold']=np.random.randint(0,1000,size=n).astype("float")/1000 #Tekrar bak ...
    adjacency_df['IsInfected']=np.zeros((n,1))
    adjacency_df['IsRisky']=np.zeros((n,1))
    adjacency_df_original = adjacency_df.copy(deep=True)
    
    dummy_vector_for_selection=np.arange(1,n+1)
    max_infected = int(infected_percentage_list[0]*n)
    max_risky = int(risky_percentage_list[0]*n)
    pool_for_all = np.random.choice(dummy_vector_for_selection,(max_infected+max_risky),replace=False)
    pool_for_infected = np.random.choice(pool_for_all,max_infected,replace=False)
    pool_for_risky = [risk for risk in pool_for_all if risk not in pool_for_infected]
    for inf_rate in infected_percentage_list:
        for risky_rate in risky_percentage_list:
            
            infected_percentage=inf_rate
            total_risky_percentage=1-infected_percentage
            risky_percentage=risky_rate

            total_infected=int(infected_percentage*n)
            total_risky=int(risky_percentage*n)
            #nx.draw(G)
            #plt.show()
            selected_infected=pool_for_infected[:total_infected]
            selected_risky=pool_for_risky[:total_risky]
            

            adjacency_df=adjacency_df_original.copy(deep=True)
            adjacency_df.loc[selected_infected,'IsInfected']=1
            adjacency_df.loc[selected_risky,'IsRisky']=1

            os.chdir("C:\\Users\\utku\\Desktop\\bitirme\\data\\adjaceny_df")
            
            adjacency_df.to_csv("adjacency_df-n"+str(n)+'-m'+str(m)+'-run'+str(elem+1)+"-risky"+str(int(risky_rate*100))+"-inf"+str(int(inf_rate*100))+"_"+str(name)+".csv",index=False)


node_features

connectivity=nx.connectivity(Di_G)

list(nx.bridges(G))

list(nx.enumerate_all_cliques(G))

from networkx.algorithms.community import greedy_modularity_communities
c = list(greedy_modularity_communities(Di_G))


c

print(setRisky)

from networkx.algorithms.community import girvan_newman


comp = girvan_newman(Di_G)

from networkx.algorithms.community.label_propagation import label_propagation_communities

comm = label_propagation_communities(G)

            setV = range(1,n+1)
            impact_matrix = adjacency_df.copy(deep=True)
            impact_matrix.loc[:,setV]=impact_matrix.loc[:,setV].div(impact_matrix.loc[:,"Threshold"],axis=0)
            impact_matrix[impact_matrix[setV] >= 1] = 1
            Di_G = nx.DiGraph(G)
            for (u, v) in G.edges():
                Di_G.edges[u,v]['weight'] = impact_matrix.iat[v,u]
                Di_G.edges[v,u]['weight'] = impact_matrix.iat[u,v]
            node_features=pd.DataFrame(np.zeros((n,1)),columns=["IsInfected"],index=range(1,n+1))
            betweenness_cent=nx.betweenness_centrality(Di_G)
            degree_cent=nx.degree_centrality(Di_G)
            closeness_cent=nx.closeness_centrality(Di_G)
            eigenvector_cent=nx.eigenvector_centrality(Di_G)
            degree=Di_G.degree()
            node_features['IsInfected']=adjacency_df['IsInfected']
            node_features['IsRisky']=adjacency_df['IsRisky']
            node_features['map_for_dict']=betweenness_cent
            node_features['Betweenness']=node_features['map_for_dict'].map(betweenness_cent)
            node_features['Degree_Cent']=node_features['map_for_dict'].map(degree_cent)
            node_features['Closeness_cent']=node_features['map_for_dict'].map(closeness_cent)
            node_features['Eigenvector_cent']=node_features['map_for_dict'].map(eigenvector_cent)
            node_features['Degree']=node_features['map_for_dict'].map(degree)
            #sets
            setS = node_features.loc[(node_features.loc[:, "IsInfected"] < 1),"IsInfected"].index.tolist()
            setP = node_features.loc[(node_features.loc[:, "IsInfected"] > 0),"IsInfected"].index.tolist()
            setRisky= node_features.loc[(node_features.loc[:, "IsRisky"] > 0),"IsRisky"].index.tolist()
            node_features['Shortest_to_Infected']=9999
            node_features['Shortest_to_Risky']=0
            for i in setS:
                shortest_inf=np.array([9999])
                shortest_risky=np.array([9999])
                for j in setP:
                    if(nx.has_path(Di_G,source=i-1,target=j-1)):
                        shortest_inf=np.append(shortest_inf,nx.shortest_path_length(Di_G,source=i-1,target=j-1))
                node_features.at[i,'Shortest_to_Infected']=shortest_inf.min()
                for k in setRisky:
                    if(nx.has_path(Di_G,source=i-1,target=k-1)):
                        shortest_risky=np.append(shortest_risky,nx.shortest_path_length(Di_G,source=i-1,target=k-1))
                node_features.at[i,'Shortest_to_Risky']=shortest_risky.min()
            node_features['Total_Impact_Ratio']=impact_matrix[setV].sum(axis=1)
            os.chdir("C:\\Users\\utku\\Desktop\\bitirme\\data\\node_features")
            #node_features.to_csv("node_features-n"+str(n)+'-m'+str(m)+'-run'+str(elem+1)+"-risky"+str(risky_rate*100)+"-inf"+str(inf_rate*100)+"_"+str(name)+".csv",index=False)
            node_features.to_csv("node_features-n"+str(n)+'-m'+str(m)+'-run'+str(elem+1)+"-risky"+str(int(risky_rate*100))+"-inf"+str(int(inf_rate*100))+"_"+str(name)+".csv",index=False)
